# ICPSR Dataset Archive

In [22]:
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual, HBox, VBox, Label

import ipywidgets as widgets
import pandas as pd
import qgrid

datasets = pd.read_json('data/data_sets.json')

In [16]:
@interact

def showDatasets(By = ['Dataset Title', 'Dataset ID'], Search="ANES 1952"):
    column = {
        'Dataset Title': 'title',
        'Dataset ID': 'data_set_id'
    }
    slist = Search.split(',')
#     return slist
    try:
        if(By == 'Dataset Title'):
            return datasets[['data_set_id', 'unique_identifier','title', 'description']].loc[datasets[column[By]].str.contains('|'.join(slist))]
        else:
            return datasets[['data_set_id', 'unique_identifier','title', 'description']].loc[datasets[column[By]].isin([int(n) for n in slist])]
    except Exception as e:
        pass
    

interactive(children=(Dropdown(description='By', options=('Dataset Title', 'Dataset ID'), value='Dataset Title…

In [3]:
import networkx as nx
import random
from networkx.readwrite import json_graph
import json
def read_json_file(filename):
    with open(filename, 'r') as f:
        js_graph = json.load(f)
    return json_graph.node_link_graph(js_graph)
def generatePairs(firstnode, allDatasets):
    return [(firstnode, i) for i in allDatasets]
def getJaccard(node, g):
    allDatasets = [i for i in g.nodes if(str(i).startswith('data'))]

    pairs = generatePairs(node, allDatasets)
    preds_jaccard = nx.jaccard_coefficient(g, pairs)
    res = []
    for u, v, p in preds_jaccard:
        if p > 0.0:
            res.append((u,v,p))
    return res

G = read_json_file('data/network_json.json')

In [31]:
btn = widgets.Button(description = "Generate")
display(Label('Press to generate random set of recommendations'), btn)
output = widgets.Output()

allDatasets = [i for i in G.nodes if(str(i).startswith('data'))]

@output.capture()
def button_handler(btn):
    sampleDataset = random.sample(allDatasets, 1)
    for i in sampleDataset:
        tmp = getJaccard(i, G)
        print("Fetching dataset ID: %s \nCalculating similarity scores for %s/%s datasets"%(i, len(tmp),len(allDatasets)))
        if len(tmp)>0:
            res = getJaccard(i, G)
            df = pd.DataFrame(res, columns=['x', 'y', 'score']).sort_values(by=['score'], ascending=False).reset_index()
            display(df[['y', 'score']].iloc[1:6])
btn.on_click(button_handler)
display(output)

Label(value='Press to generate random set of recommendations')

Button(description='Generate', style=ButtonStyle())

Output()